# comercio_ext_indices.tb_referencia_ncm_ppi
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_PPI_delta`
## 📌 Descrição do arquivo

Classificação **PPI** (PT/EN) para traduzir o campo `CO_PPI` da dimensão NCM.

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_PPI`|Código PPI|Identificador da categoria PPI|
|`NO_PPI`|Nome (PT)|Descrição do PPI em português|
|`NO_PPI_MIN`|Nome (PT – minúsculo)|Variante textual|
|`NO_PPI_ING`|Nome (EN)|Descrição em inglês|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
import org . apache . spark  . sql . functions . _
import org . apache . spark  . sql . types . _
import org . apache . spark  . sql . DataFrame
import org . apache . hadoop . fs  . FileSystem 
import org . apache . hadoop . fs  . Path 
import io  . delta  . tables . _

In [0]:

val bronzePath =     "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_PPI_delta/"   
val silverPath =     "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_ppi/"   
val silverTable =    "tb_referencia_ncm_ppi"
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)



In [0]:

val silverSchema = StructType(
    Seq (
      StructField ( "CO_PPI" , StringType ,  nullable = false ) ,
      StructField ( "NO_PPI" , StringType ,  nullable = false ) ,
      StructField ( "NO_PPI_MIN" , StringType ,  nullable = false ) ,
      StructField ( "NO_PPI_ING" , StringType ,  nullable = false ) ,
      StructField ( "TS_REF" , TimestampType ,  nullable = false ) ,
      StructField ( "NM_ORIGEM" , StringType ,  nullable = false ) 
    )
  )

## Extração
> #### **saprk.read**

In [0]:
val dfBronzeRaw = spark . read . format ( "delta" ) . load ( bronzePath )

##Normalização
> #### **datatype**
> #### **regras**

In [0]:
val dfNormalized = dfBronzeRaw
  .withColumn("CO_PPI", trim(upper(col("CO_PPI").cast(StringType))))
  .withColumn("NO_PPI", col("NO_PPI").cast(StringType))
  .withColumn("NO_PPI_MIN", col("NO_PPI_MIN").cast(StringType))
  .withColumn("NO_PPI_ING", col("NO_PPI_ING").cast(StringType))

In [0]:
val dfWithDefaults = dfNormalized
  .withColumn("TS_REF", current_timestamp())
  .withColumn("NM_ORIGEM", lit("/landingbeca2026jan/balancacomercial/NCM_PPI_delta"))

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:

val dfValid = dfWithDefaults . filter ( 
    col("CO_PPI").isNotNull
  )


In [0]:
val dfDedup = dfValid . dropDuplicates (
    Seq (
        "CO_PPI"
    )
  )

In [0]:
val dfSilver = dfDedup.select( col("CO_PPI"), col("NO_PPI"), col("NO_PPI_MIN"), col("NO_PPI_ING"), col("TS_REF"), col("NM_ORIGEM") )

##Carga
> #### **overwrite**

In [0]:
spark.conf.set("spark.databricks.optimizer.dynamicPartitionPruning", "true")
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

val silverExists = fs.exists(new Path(silverPath))

if (!silverExists) {
  dfSilver
    .write
    .format("delta")
    .mode("overwrite")
    .save(silverPath)
}